## 3.3 线性回归的简洁实现

torch.utils.data模块提供了有关数据处理的工具  
torch.nn模块定义了大量神经网络的层  
torch.nn.init模块定义了各种初始化方法  
torch.optim模块提供了很多常用的优化算法

In [1]:
import matplotlib_inline
import torch
import torch.utils.data as Data
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
from IPython import display
from IPython.display import set_matplotlib_formats
from matplotlib import pyplot as plt
import numpy as np
import random

In [2]:
# 同上一章生成数据
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [3]:
# 代替上一章的data_iter，即洗牌然后拿数据
batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.5796,  0.5871],
        [-0.5142,  0.3326],
        [-0.2489, -0.5346],
        [ 0.1567, -1.3743],
        [ 1.0427, -2.2903],
        [ 0.0211,  0.7242],
        [ 2.1995,  0.5481],
        [ 0.0609,  1.0279],
        [-0.0235,  0.9853],
        [ 1.5511,  2.0897]]) tensor([ 3.3771,  2.0198,  5.5311,  9.1775, 14.0875,  1.7849,  6.7429,  0.8279,
         0.8068,  0.2111])


In [4]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)

    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

# net = LinearNet(num_inputs)
# 使用print可以打印出网络的结构
# print(net)


# 但推荐以下写法，在网络带有流程的时候会好用一些
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
)
print(net)
print(net[0])

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [5]:
# 可以通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器。
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.4267, 0.2883]], requires_grad=True)
Parameter containing:
tensor([-0.6014], requires_grad=True)


In [6]:
# 初始化
init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

# 损失函数
loss = nn.MSELoss()

# 最优化方法
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [7]:
# 训练模型
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()  # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

epoch 1, loss: 0.000063
epoch 2, loss: 0.000046
epoch 3, loss: 0.000090
[2, -3.4] Parameter containing:
tensor([[ 2.0002, -3.4010]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1998], requires_grad=True)
